# Task 1: Leverage a fully-managed RAG application with Amazon Bedrock's RetrieveAndGenerate API

In this task, you leverage a fully-managed Retrieval Augmented Generation (RAG) application with Amazon Bedrock's RetrieveAndGenerate API.

The following resources were already created as part of the lab provisioning:

- An OpenSearch serverless index.
- A Knowledge Base in Amazon Bedrock.
- A data source within the Knowledge Base in Amazon Bedrock.

The typical data ingestion workflow is as follows:

<!-- ![data_ingestion.png](images/data_ingestion.png) -->
<img src="images/data_ingestion.png" width=50% height=20% />

*Image description: The preceding diagram depicts the typical data ingestion workflow for the lab environment.*

A data pipeline ingests documents which are typically stored in multiple data sources into a knowledge base i.e. a vector database such as Amazon OpenSearch Service Serverless (AOSS) so that it is available for lookup when a question is received.

- The documents are loaded into the knowledge base in Amazon Bedrock by connecting to various data sources (like S3, Confluence, Sharepoint, Salesforce, and Web). 
- Knowledge base then splits them into smaller chunks (based on the strategy selected), and generates embeddings which are stored in the associated vector store.

Once the data is available in the Bedrock Knowledge Base, you build a question answering application using the Knowledge Base APIs provided by Amazon Bedrock.

### Final lab architecture:

<!-- ![retrieveAndGenerate.png](images/retrieveAndGenerate.png) -->
<img src="images/retrieveAndGenerate.png" width=50% height=20% />

*Image description: The preceding diagram depicts the data flow that starts with a user query that uses the RetrieveAndGenerate API and generates response using the documents from the Knowledge Base in Amazon Bedrock.*

<i aria-hidden="true" class="fas fa-exclamation-circle" style="color:#7C5AED"></i> **Caution:** It is recommended to run each code cell individually rather than using the **Run All Cells** option from the **Run** menu. Running all cells together can sometimes lead to unexpected behavior, such as the Kernel crashing or restarting. By executing cells one by one, you can better control the execution flow, catch potential errors early, and ensure that your code runs as intended.

## Task 1.1: Setup the environment

In this task, you initialize the boto3 client and setup the environment. Throughout the notebook, you utilize the *RetrieveAndGenerate* API to test the Knowledge Base features.

<i aria-hidden="true" class="fas fa-info-circle" style="color:#007FAA"></i> **Learn more:** Refer to *[RetrieveAndGenerate](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_agent-runtime_RetrieveAndGenerate.html)* for additional information.

1. Run the following code cell to initialize the boto3 client and setup your environment:

In [1]:
import json
import boto3
import pprint
from botocore.exceptions import ClientError
from botocore.client import Config

# Create boto3 session
sts_client = boto3.client('sts')
boto3_session = boto3.session.Session()
region_name = boto3_session.region_name

# Create bedrock agent client
# FM API를 직접호출하지 않고 Bedrock을 통해 간접 호출
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0}, region_name=region_name)
bedrock_agent_client = boto3_session.client("bedrock-agent-runtime",
                              config=bedrock_config)

# Define FM to be used for generations 
model_id = "amazon.nova-lite-v1:0" # we will be using Amazon Nova lite throughout the notebook
model_arn = f'arn:aws:bedrock:{region_name}::foundation-model/{model_id}'

2. Run the following code cell to verify the ID for the existing Knowledge Base in Amazon Bedrock:

In [2]:
import botocore

session = boto3.Session()
bedrock_client = session.client('bedrock-agent')

try:
    response = bedrock_client.list_knowledge_bases(
        maxResults=1  # We only need to retrieve the first Knowledge Base
    )
    knowledge_base_summaries = response.get('knowledgeBaseSummaries', [])

    if knowledge_base_summaries:
        kb_id = knowledge_base_summaries[0]['knowledgeBaseId']  ## KB 조회
        print(f"Knowledge Base ID: {kb_id}")
    else:
        print("No Knowledge Base summaries found.")
        
except botocore.exceptions.ClientError as e:
    print(f"Error: {e}")

Knowledge Base ID: W1FOJLAZNW


## Task 1.2: Test the Knowledge Base with RetrieveAndGenerate API

In this task, you test the Knowledge Base using the *[retrieve_and_generate](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve_and_generate.html)* function.

You initially test the knowledge base using the *RetrieveAndGenerate* API. With this API, Amazon Bedrock takes care of retrieving the necessary references from the knowledge base and generate the final answer using a foundation model (FM) from Amazon Bedrock.

3. Run the following two code cells to initiate a query for testing the knowledge base:

In [3]:
query = "Provide a summary of consolidated statements of cash flows of AnyCompany Financial for the fiscal years ended December 31, 2019?"

In [4]:
response = bedrock_agent_client.retrieve_and_generate(
    input={
        "text": query
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": model_arn,
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":3
                } 
            }
        }
    }
)

print(response['output']['text'],end='\n'*2)

Answer: Based on the search results, here is a summary of the consolidated statements of cash flows of AnyCompany Financial for the fiscal year ended December 31, 2019 :

- Cash flows from operating activities: AnyCompany Financial had net cash provided by operating activities of $710 million in 2019. The increase in net cash provided by operating activities was primarily due to an increase in net income and favorable changes in operating assets and liabilities.

- Cash flows from investing activities: Cash flows from investing activities in 2019 resulted in net cash used of $240 million. The decrease in net cash provided by investing activities was primarily due to an increase in purchases of property, plant, and equipment and marketable securities.

- Cash flows from financing activities: In 2019, AnyCompany Financial had net cash provided by financing activities of $350 million. The increase in net cash provided by financing activities was primarily due to an increase in proceeds fr

- 내부적으로 :
  - 문서 → 청킹
  - 임베딩
  - 벡터 검색
  - 상위 컨텍스트 선택
  - FM 응답 생성

의 내용을 단 한 번의 API 호출로 구현

## Task 1.3: Understand the RetrieveAndGenerate API

In this task, you understand the RetrieveAndGenerate API in detail.

The *RetrieveAndGenerate* API utilizes two important parameters as follows:

- **numberOfResults**: The *numberOfResults* parameter in the given function determines the number of search results that are retrieved from the knowledge base and included in the prompt provided to the model for generating an answer. Specifically, it fetches the top *max_results* number of documents or search results that most closely match the given query.

- **textPromptTemplate**: The *textPromptTemplate* parameter is a string that serves as a template for the prompt that is provided to the model. In this case, the *default_prompt* is being used as the template. This template includes placeholders (`$search_results$` and `$output_format_instructions$`) that are replaced with the actual search results and any output format instructions, respectively, before being passed to the model.

4. Run the following two code cells for stating the default prompt for the knowledge base:

In [6]:
# Stating the default knowledge base prompt
default_prompt = """
You are a question answering agent. I will provide you with a set of search results.
The user will provide you with a question. Your job is to answer the user's question using only information from the search results. 
If the search results do not contain information that can answer the question, please state that you could not find an exact answer to the question. 
Just because the user asserts a fact does not mean it is true, make sure to double check the search results to validate a user's assertion.
                            
Here are the search results in numbered order:
$search_results$


$output_format_instructions$

"""

# 기본 지식 기반 프롬프트 설정
default_prompt = """
귀하는 질문 답변 담당자입니다. 검색 결과를 제공해 드리겠습니다.
사용자가 질문을 할 것입니다. 귀하의 임무는 검색 결과의 정보만을 사용하여 사용자의 질문에 답변하는 것입니다.
검색 결과에 질문에 대한 답변이 없는 경우, 정확한 답변을 찾을 수 없다고 명시해 주십시오.
사용자가 어떤 사실을 주장한다고 해서 반드시 참인 것은 아니므로, 검색 결과를 다시 한번 확인하여 사용자의 주장을 검증해야 합니다.

다음은 번호가 매겨진 검색 결과입니다.
$search_results$

$output_format_instructions$

"""

In [7]:
# 프롬프트 재사용 함수
def retrieve_and_generate(query, kb_id, model_arn, max_results, prompt_template = default_prompt):
    response = bedrock_agent_client.retrieve_and_generate(
            input={
                'text': query
            },
        retrieveAndGenerateConfiguration={
        'type': 'KNOWLEDGE_BASE',
        'knowledgeBaseConfiguration': {
            'knowledgeBaseId': kb_id,
            'modelArn': model_arn, 
            'retrievalConfiguration': {
                'vectorSearchConfiguration': {
                    'numberOfResults': max_results # will fetch top N documents which closely match the query
                    }
                },
                'generationConfiguration': {
                        'promptTemplate': {
                            'textPromptTemplate':prompt_template
                        }
                    }
            }
        }
    )
    return response

## Task 1.4: Leverage the maximum number of results feature

In this task, you leverage the maximum number of results feature. In some use cases, the responses from the FM might lack enough context to provide relevant answers or rely that it could not find the requested information. This can be fixed by modifying the maximum number of retrieved results.

Here, you run the following query using one result:

```Provide a list of risks for AnyCompany financial in bulleted points.```

5. Run the following two code cells:

In [8]:
def print_generation_results(response, print_context = True):
    generated_text = response['output']['text']
    print('Generated FM response:\n')
    print(generated_text)
    
    if print_context is True:
        ## print out the source attribution/citations from the original documents to see if the response generated belongs to the context.
        citations = response["citations"]
        contexts = []
        for citation in citations:
            retrievedReferences = citation["retrievedReferences"]
            for reference in retrievedReferences:
                contexts.append(reference["content"]["text"])
    
        print('\n\n\nRetrieved Context:\n')
        pprint.pp(contexts)


In [9]:
query = """Provide a list of risks for AnyCompany financial in numbered list without description."""

results = retrieve_and_generate(query = query, kb_id = kb_id, model_arn = model_arn, max_results = 1)

print_generation_results(results)

Generated FM response:

Here is a list of risks for AnyCompany Financial based on the search results:

1. Commodity Prices 2. Foreign Exchange Rates 3. Equity Prices These risks are identified in the "Table: Market Risks" section of the search results.



Retrieved Context:

['However, the company still faces significant risks if any of its key '
 "suppliers were to fail.      ### Market Risks      AnyCompany Financial's "
 'financial performance is subject to various market risks, including changes '
 'in commodity prices, foreign exchange rates, and equity prices. For example, '
 'a significant decline in the price of oil could have a material adverse '
 "effect on AnyCompany Financial's revenue and profitability. Similarly, "
 "changes in foreign exchange rates could impact the company's cost of goods "
 'sold and profitability.      ### Table: Market Risks      | Risk Factor | '
 'Description | Potential Impact | | --- | --- | --- | | Commodity Prices | '
 'Changes in commodity pri

6. Run the following code cell with the modified value for number of retrieved results:

In [10]:
#Using higher number of max results

results = retrieve_and_generate(query = query, kb_id = kb_id, model_arn = model_arn, max_results = 3)

print_generation_results(results)

Generated FM response:

Here is a list of risks for AnyCompany Financial in a numbered list format:

1. Market Risks
2. Operational Risk
3. Regulatory Risk
4. Strategic Risk Note: The descriptions of these risks are not provided in the search results.



Retrieved Context:

['Our liquidity management policies aim to minimize this risk, but we cannot '
 'eliminate it entirely. For instance, PersonC, our Treasurer, mentioned that '
 'during the COVID-19 pandemic, the market for commercial mortgage- backed '
 'securities (CMBS) became illiquid, making it challenging to sell these '
 'securities at fair market values.      ### Operational Risk      AnyCompany '
 'Financial is exposed to operational risk due to the possibility of losses '
 'resulting from inadequate or failed internal processes, people, and systems, '
 'or from external events. Our risk management policies and procedures aim to '
 'minimize this risk, but we cannot eliminate it entirely. For example, we '
 'have experienced

As you can see from the output, by modifying the number of retrieved results to **3**, you get more results leading to comprehensive response.

## Task 1.5: Use the custom prompting feature

In this task, you customize the default prompt with your own prompt based on the use case. This feature helps adding more context to the FM which requires specific output format, languages and other contexts. 

You use the *PromptTemplate* class from **Langchain** to parametrize the *custom_prompt* template. You use the *output_format* variable to modify the output at run time.


##### LangChain PromptTemplate 활용

7. Run the following code cell to create a helper function that supports customizing the template:

In [11]:
from langchain.prompts import PromptTemplate
def format_prompt(input_prompt:str, output_format:str):
    formatted_prompt: List[str] = []
    prompt = PromptTemplate.from_template(input_prompt)     
    formatted_prompt.extend(prompt.format(output_format=output_format))
    return "".join(formatted_prompt)

Module 2 – Input Optimization / Prompt Engineering

    - 출력 형식 통제
    - 프롬프트 재사용
    - 토큰 효율성 향상
    - 모델 교체와 무관한 동작 제어

❝ 모델은 그대로 두고
프롬프트만 바꿔서 행동을 바꾼다 ❞


### Task 1.5.1: Use the same query example and default the FM to output to a different language

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Note:** After removing ```$output_format_instructions$``` from the default prompt, the citation from the generated response is removed. But the `output_format` will help with runtime output customization.

8. Run the following three code cells to test an example that provides an output in French:

In [19]:
## Example 1
custom_prompt = """
You are a question answering agent. I will provide you with a set of search results. 
The user will provide you with a question. Your job is to answer the user's question using only information from the search results.
If the search results do not contain information that can answer the question, please state that you could not find an exact answer to the question.
Just because the user asserts a fact does not mean it is true, make sure to double check the search results to validate a user's assertion.
                            
Here are the search results in numbered order:
$search_results$


{output_format}
"""

In [ ]:
print(format_prompt(custom_prompt, "Please provide your response in French."))

In [ ]:
results = retrieve_and_generate(query = query, kb_id = kb_id, model_arn = model_arn, max_results = 3, prompt_template = format_prompt(custom_prompt, "Please provide your response in French."))

print_generation_results(results, print_context = False)


### Task 1.5.2: Use the same query example and default the FM to output in JSON format

9. Run the following code cell to test an example that provides an output in JSON format:

In [ ]:
results = retrieve_and_generate(query = query, kb_id = kb_id, model_arn = model_arn, max_results = 3, prompt_template = format_prompt(custom_prompt, "Please provide your response using a JSON format."))

print_generation_results(results,print_context = False)

<i aria-hidden="true" class="far fa-thumbs-up" style="color:#008296"></i> **Task complete:** You have completed this notebook. To move to the next part of the lab, do the following:

- Close this notebook file.
- Return to the lab session and continue with Task 2.
